# S1

In [1]:
# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.sentinelhub import SentinelHub

# xcube imports
from xcube.core.geom import mask_dataset_by_geometry

# Various utilities
from datetime import date
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
catchment_outline = gpd.read_file('catchment_outline.geojson')

In [3]:
SH = SentinelHub()
SH.dataset_names

['LOTL1',
 'LTML2',
 'LMSSL1',
 'LTML1',
 'S1GRD',
 'S5PL2',
 'CUSTOM',
 'S2L1C',
 'HLS',
 'S3OLCI',
 'DEM',
 'S2L2A',
 'S3SLSTR',
 'LETML1',
 'MODIS',
 'LOTL2',
 'LETML2']

In [4]:
SH.band_names('S1GRD')

['VV', 'HH', 'VH', 'localIncidenceAngle', 'scatteringArea', 'shadowMask', 'HV']

In [12]:
bbox = catchment_outline.bounds.iloc[0]

start_date = date(2017, 12, 1)
end_date = date(2018, 1, 31)

cube_config = CubeConfig(
    dataset_name='S1GRD',
    band_names=['VV', 'VH', 'localIncidenceAngle'],
    bbox=bbox.tolist(),
    spatial_res=0.0018,   # = 100 meters in degree>
    time_range=[start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")],
    time_tolerance='6D'
)

In [13]:
cube = open_cube(cube_config, api_url="https://creodias.sentinel-hub.com")
cube

<xarray.Dataset>
Dimensions:              (time: 21, lat: 167, lon: 192, bnds: 2)
Coordinates:
  * lat                  (lat) float64 46.95 46.95 46.95 ... 46.66 46.66 46.65
  * lon                  (lon) float64 11.02 11.02 11.03 ... 11.36 11.36 11.37
  * time                 (time) datetime64[ns] 2017-12-01T05:17:50 ... 2018-0...
    time_bnds            (time, bnds) datetime64[ns] dask.array<chunksize=(21, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    VH                   (time, lat, lon) float32 dask.array<chunksize=(1, 167, 192), meta=np.ndarray>
    VV                   (time, lat, lon) float32 dask.array<chunksize=(1, 167, 192), meta=np.ndarray>
    localIncidenceAngle  (time, lat, lon) float32 dask.array<chunksize=(1, 167, 192), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   S1GRD Data Cube Subset
    history:                 [{'program': 'xcube_sh.chunkstore.SentinelHubChu...
    date_created:            2023-02-21T09:40:18.783677
    time_coverage_start:     2017-12-01T05:17:50+00:00
    time_coverage_end:       2018-01-30T05:17:47+00:00
    time_coverage_duration:  P59DT23H59M57S
    geospatial_lon_min:      11.020833333333357
    geospatial_lat_min:      46.653599378797765
    geospatial_lon_max:      11.366433333333356
    geospatial_lat_max:      46.95419937879777
    processing_level:        L1B

In [14]:
cube.time

<xarray.DataArray 'time' (time: 21)>
array(['2017-12-01T05:17:50.000000000', '2017-12-06T05:26:04.000000000',
       '2017-12-07T05:18:25.000000000', '2017-12-12T05:26:34.000000000',
       '2017-12-13T05:17:49.000000000', '2017-12-18T05:26:03.000000000',
       '2017-12-19T05:18:25.000000000', '2017-12-24T05:26:33.000000000',
       '2017-12-25T05:17:49.000000000', '2017-12-30T05:26:03.000000000',
       '2017-12-31T05:18:24.000000000', '2018-01-05T05:26:33.000000000',
       '2018-01-06T05:17:48.000000000', '2018-01-11T05:26:02.000000000',
       '2018-01-12T05:18:24.000000000', '2018-01-17T05:26:33.000000000',
       '2018-01-18T05:17:48.000000000', '2018-01-23T05:26:02.000000000',
       '2018-01-24T05:18:23.000000000', '2018-01-29T05:26:32.000000000',
       '2018-01-30T05:17:47.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-12-01T05:17:50 ... 2018-01-30T05:17:47
Attributes:
    standard_name:  time
    bounds:         time_bnds

Only acquisition in morning are available (descending) no afternoon acquisition are available in the afternoon (ascending)

We need to filter for ASCENDING only. Better for track number.